In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector
import requests
import time
import re

from selenium import webdriver
from unidecode import unidecode

from selenium.webdriver.support import expected_conditions as EC

In [5]:
def criar_conexao(host, usuario, password, banco):
    return mysql.connector.connect(
        host=host,
        user=usuario,
        password=password,
        database=banco
    )

def fechar_conexao(conn):
    return conn.close()

def insert_banco(tipo_feriado, data, nome_feriado, municipio, estado):
    try:
        conn = criar_conexao('localhost', 'root', 'xxxxxxx', 'api_feriado')
        cursor = conn.cursor()
        
        sql = """INSERT INTO feriados_municipio (tipo_feriado, data, nome_feriado, municipio, estado)
                VALUES (%s, %s, %s, %s, %s)"""
        
        valor = (tipo_feriado, data, nome_feriado, municipio, estado)


        cursor.execute(sql, valor)
        
        conn.commit()
        cursor.close()
        
        fechar_conexao(conn)
        
    except mysql.connector.Error as err:
    
        print(f"Ocorreu um erro no banco de dados: {err}")
        
        

In [6]:
# Configurar o driver do Selenium
driver = webdriver.Chrome()

states = [
    'ac', 'al', 'ap', 'am', 'ba', 'ce', 'df', 'es', 'go', 'ma', 'mt',
    'ms', 'mg', 'pa', 'pb', 'pr', 'pe', 'pi', 'rj', 'rn', 'rs', 'ro',
    'rr', 'sc', 'sp', 'se', 'to'
]

# Encontrar o elemento <select> pelo ID
for ano in range(2021, 2030):
    time.sleep(120)

    for state in states:
        url = f'https://www.feriados.com.br/feriados-estado-{state}.php?ano={ano}'

        driver.get(url)
        select_element = driver.find_elements('xpath', '//*[@id="cidade"]')
        municipios = unidecode(select_element[0].text.lower().replace(' ', '_'))
        municipios = municipios.split("\n")
        municipios.pop(0)

        for municipio in municipios:
                print(f'Localização: {municipio}-{state} - {ano}')
                
                ws_url = f'{municipio}-{state}'
                
                url = f'https://www.feriados.com.br/feriados-{ws_url}.php?ano={ano}'
        
                response = requests.get(url)

                content = response.content

                site = BeautifulSoup(content, 'html.parser')
                
                # Localization city -------------------
                name_location = site.find('div', attrs={'class': 'rounded_borders'})

                title = name_location.find('h2')

                title_holiday = title.get_text()

                first_space = title_holiday.find(' ')
                last_space = title_holiday.rfind(' ')

                title_holiday = title_holiday[first_space+1:last_space]
                
                
                # Holiday table -------------------
                holidays_div = site.select('ul.multi-column div')

                data_holiday = []

                for div in holidays_div:
                    type_holiday = div.get('title')
                    type_holiday = re.sub(r'<.*?>', '', type_holiday).strip()

                    text_holiday = div.get_text(strip=True)

                    if ' - ' in text_holiday:
                        date_holiday, name_holiday = text_holiday.split(' - ', 1)
                    else:
                        date_holiday = None
                        name_holiday = text_holiday

                    info = {
                        'tipo_feriado': type_holiday,
                        'data': date_holiday,
                        'nome_feriado': name_holiday,
                        'municipio': title_holiday,
                        'estado': state.upper()
                    }

                    data_holiday.append(info)

                # Data processing
                for data in data_holiday:
                    if data['data'] is None:
                        data['data'] = data['nome_feriado'][:10]
                        data['nome_feriado'] = data['nome_feriado'][12:]

                    elif len(data['data']) > 10:
                        data['data'] = data['data'][:10]

                    data['data'] = pd.to_datetime(data['data'], format='%d/%m/%Y')
                    

                for data in data_holiday:
                    try:
                        insert_banco(data['tipo_feriado'], data['data'], data['nome_feriado'], data['municipio'], data['estado'])
                    
                    except:
                        print(f"""Erro ao subir os dados:\n
                              {data['tipo_feriado']} - {data['data']} - 
                              {data['nome_feriado']} - {data['municipio']} -
                              {data['estado']}
                              """)

    
print('--------- Finalizou!!! ---------') 
                
            

Localização: acrelandia-ac - 2021
Localização: assis_brasil-ac - 2021
Localização: brasileia-ac - 2021
Localização: bujari-ac - 2021
Localização: capixaba-ac - 2021
Localização: cruzeiro_do_sul-ac - 2021
Localização: epitaciolandia-ac - 2021
Localização: feijo-ac - 2021
Localização: jordao-ac - 2021
Localização: mancio_lima-ac - 2021
Localização: manoel_urbano-ac - 2021
Localização: marechal_thaumaturgo-ac - 2021
Localização: placido_de_castro-ac - 2021
Localização: porto_acre-ac - 2021
Localização: porto_walter-ac - 2021
Localização: rio_branco-ac - 2021
Localização: rodrigues_alves-ac - 2021
Localização: santa_rosa_do_purus-ac - 2021
Localização: sena_madureira-ac - 2021
Localização: senador_guiomard-ac - 2021
Localização: tarauaca-ac - 2021
Localização: xapuri-ac - 2021
Localização: agua_branca-al - 2021
Localização: anadia-al - 2021
Localização: arapiraca-al - 2021
Localização: atalaia-al - 2021
Localização: barra_de_santo_antonio-al - 2021
Localização: barra_de_sao_miguel-al - 202

# ----------------------------------------------------------------